In [1]:
import glob
import re

# taking stanford-parsed texts as tokens
# why? because LinkAppend requires tokenised sentences and we want the results to be comparable to other models
# plus, the parser is quite good
basepath = '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/'


In [2]:
def extract_sentences_from_text(text):
    # regex specific to stanford output format
    sentence_pattern = re.compile(r'Tokens:\n(.*?)\n\nConstituency parse:', re.DOTALL)
    token_pattern = re.compile(r'Text=([^\s\[]+)', re.DOTALL)

    matches = sentence_pattern.findall(text)
    reconstructed_sentences = []
    for tokens_text in matches:
        matches = token_pattern.findall(tokens_text)
        sentence = ' '.join(matches)
        reconstructed_sentences.append(sentence)
    return reconstructed_sentences

In [3]:
parsed_files = glob.glob(basepath + '*.out')
parsed_files = [f for f in parsed_files if 'checkpoint' not in f]

In [4]:
parsed_files

['/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_261.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_316.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_1373.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_1501.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_1755.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_1865.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_2232.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_2392.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/stanford-det/doc_3224.out',
 '/home/xilini/coref/multimodal_coref/data/nikolai_test_out/random

In [5]:
import tqdm

vwp_linkappend_texts = []

for file in tqdm.tqdm(parsed_files):

    with open(file, 'r') as f:
        content = f.read()
    sentences = extract_sentences_from_text(content)


    #sentences = ["It 's a picture of what look like washing machines . There are three of them in a row , plus one stacked on top .",\
    #             'A big blue bag is hanging from the top right washing machine .',\
    #             'There are four large silver pipes / tubes coming out of the wall and running behind the machines .',\
    #             "There 's a pile of clothes stacked on top of the two left washing machines . They all have clear doors so you can see there 's also clothing inside them .",\
    #             "The angle of the picture means you ca n't see the floor of the room ."]

    #sentences = ["This a dining room , an interior room in a house or hotel suite .",\
    #             'On each side of the room , there are arched passages , the one on the right is doorway into a sitting room .',\
    #             'The other side is an arched opening , with a half - wall forming a shelf . Some flowers or potted plants are sitting on the shelf .',\
    #             "The ceiling is pretty high , but it looks like a soffit or coffered ceiling with a pair of steps that form rectangles .",\
    #             "In the middle of the room is a smaller dining table with a total of six chairs around it . The chairs and the table are made of dark stained wood ."]

    processed_data = []
    for i, sentence in enumerate(sentences, 1):
        words = sentence.split()
        tokens = [{'id': idx+1, 'text': word} for idx, word in enumerate(words)]
        processed_data.append({
            'id': i,
            'speaker': None,
            'text': sentence,
            'tokens': tokens
        })

    text_dict = {}
    text_dict['id'] = 'doc_' + str(file.split('_')[-1].split('.out')[0])
    text_dict['sentences'] = processed_data
    text_dict['coref_chains'] = None
    
    vwp_linkappend_texts.append(text_dict)

100%|████████████████████████████████████████████████| 20/20 [00:00<00:00, 3096.34it/s]


In [6]:
import json
with open('/home/xilini/coref/multimodal_coref/data/nikolai_test_out/randomsample_20/link-append/inputs_random20.json', 'w') as f:
    json.dump(vwp_linkappend_texts, f)

In [6]:
import json
with open('/home/xilini/tell-me-more-coref/test/inputs_random20.json', 'w') as f:
    json.dump(vwp_linkappend_texts, f)